<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.10.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: dd79ad5dfd349dc3b1ae00a307ee636a9ab6b2ef

pandas    : 1.5.3
matplotlib: 3.7.2
openai    : 0.28.1
numpy     : 1.23.5
tqdm      : 4.66.1
json      : 2.0.9
termcolor : 2.3.0
watermark : 2.4.2



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

We start by getting a list of supported models.

In [5]:
model_list = openai.Model.list()["data"]

In total we have 25 models

In [6]:
len(model_list)

26

Along with some information about each model...

In [7]:
model_list[:3]

[<Model model id=gpt-3.5-turbo-0301 at 0x7f7c988bb9c0> JSON: {
   "id": "gpt-3.5-turbo-0301",
   "object": "model",
   "created": 1677649963,
   "owned_by": "openai"
 },
 <Model model id=text-embedding-3-large at 0x7f7c988ef7e0> JSON: {
   "id": "text-embedding-3-large",
   "object": "model",
   "created": 1705953180,
   "owned_by": "system"
 },
 <Model model id=dall-e-3 at 0x7f7c988ef1f0> JSON: {
   "id": "dall-e-3",
   "object": "model",
   "created": 1698785189,
   "owned_by": "system"
 }]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model['id'] for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo-preview
gpt-4-vision-preview
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
%%time
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 3.25 ms, sys: 1.79 ms, total: 5.04 ms
Wall time: 1.78 s


Which produces a response object

In [10]:
type(response)

openai.openai_object.OpenAIObject

Which we can treat as a JSON object

In [11]:
pprint(response)

<OpenAIObject chat.completion id=chatcmpl-8nDieCs8VMlFOsJHFmKWfJCgxMDLT at 0x7f7cd0c26110> JSON: {
  "id": "chatcmpl-8nDieCs8VMlFOsJHFmKWfJCgxMDLT",
  "object": "chat.completion",
  "created": 1706741500,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Superman's main weakness is Kryptonite, a mineral from his home planet of Krypton. Exposure to Kryptonite weakens and eventually kills Superman, as it effectively removes his superhuman powers and renders him vulnerable. Other weaknesses include red sun radiation, magic, and psychic attacks, but Kryptonite is the most commonly known weakness of the character."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 72,
    "total_tokens": 85
  },
  "system_fingerprint": null
}


The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response["choices"][0]["message"]["content"]

"Superman's main weakness is Kryptonite, a mineral from his home planet of Krypton. Exposure to Kryptonite weakens and eventually kills Superman, as it effectively removes his superhuman powers and renders him vulnerable. Other weaknesses include red sun radiation, magic, and psychic attacks, but Kryptonite is the most commonly known weakness of the character."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [13]:
%%time
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 4.04 ms, sys: 1.68 ms, total: 5.72 ms
Wall time: 7.05 s


And we can access each of the answers individually int he choices list

In [14]:
for output in response["choices"]:
    print("==========")
    print(output["message"]["role"].title()) 
    print("==========")
    print(output["message"]["content"])
    print("==========\n")

Assistant
In the DC Comics universe, Kryptonite is a fictional mineral from Superman's home planet of Krypton that has detrimental effects on him and other Kryptonians. There are several different kinds of Kryptonite, each with its own unique properties and effects. Here are some of the notable types:

1. Green Kryptonite: The most well-known and common form of Kryptonite, green Kryptonite weakens and can eventually kill Superman. It negates his powers, causes extreme pain, and prolonged exposure can be fatal.

2. Red Kryptonite: Red Kryptonite has unpredictable effects on Superman and temporarily alters his behavior. Its effects can vary from enhancing or suppressing his powers to changing his personality or physical appearance.

3. Blue Kryptonite: Blue Kryptonite affects Bizarro, an imperfect clone of Superman, rather than the original Superman. It weakens and causes pain to Bizarro, just like green Kryptonite weakens Superman.

4. Gold Kryptonite: Gold Kryptonite permanently remove

In [15]:
response["usage"]

<OpenAIObject at 0x7f7d0060cfe0> JSON: {
  "prompt_tokens": 17,
  "completion_tokens": 1109,
  "total_tokens": 1126
}

# Temperature

In [16]:
%%time
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 5.02 ms, sys: 2.17 ms, total: 7.19 ms
Wall time: 16.9 s


In [17]:
print(response["choices"][0]["message"]["content"])

Once upon a time in a small village, there lived a farmer named John. John was known for his hard work and dedication towards his crops. His farm was known across surrounding areas for the bountiful harvest he produced every year. However, John faced a serious problem that haunted him every year – naughty monkeys!

These mischievous monkeys loved stealing vegetables from John's farm whenever he turned his back. The villagers would witness several monkeys swinging from trees, chattering as if orchestrating their heists.

Fed up with his crops being neither small nor protected, John decided to protect his farm from the curious monkeys. He burned the midnight oil and created various scarecrows, hoping they would deter these little thieves from his garden.

Early the next day, observing a group of monkeys approach his field, John peacefully watched from a hidden perch. As soon as the monkeys unwarily stepped raided near the discovered vegetable patch, they struggled with low lingering supe

In [18]:
%%time
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 3.28 ms, sys: 1.75 ms, total: 5.03 ms
Wall time: 8.4 s


In [19]:
print(response["choices"][0]["message"]["content"])

Once upon a time, in a small village nestled in the heart of a lush forest, there lived a young girl named Lily. She was known for her kind heart and adventurous spirit. One sunny morning, as she was exploring the woods, she stumbled upon a wounded bird with a broken wing.

Lily carefully picked up the bird and cradled it in her hands. She knew she had to help it, so she decided to take it home. With great care, she fashioned a small nest for the bird and gently placed it inside. Lily named the bird Ruby, after its vibrant red feathers.

Days turned into weeks, and Lily tirelessly tended to Ruby's needs. She fed her, kept her warm, and even sang to her when she felt lonely. Slowly but surely, Ruby's wing began to heal, and she regained her strength.

One day, as Lily was preparing to release Ruby back into the wild, a sudden storm swept through the village. The wind howled, and rain poured down in torrents. Lily worried that Ruby wouldn't survive the harsh weather, so she made a quick 

# Function Calls

In [20]:
def chat(messages, functions):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [21]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [22]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [23]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

In [24]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)

In [25]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Which location would you like to know the weather for?



Once we provide the missing information, it will generate the appropriate function arguments for us.

In [26]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [27]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)

In [28]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Which location would you like to know the weather for?

user: I'm in New York, NY.

assistant: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York, NY\",\n  \"format\": \"celsius\"\n}"
}



In [29]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': "What's the weather like today"},
 <OpenAIObject at 0x7f7d0060f650> JSON: {
   "role": "assistant",
   "content": "Which location would you like to know the weather for?"
 },
 {'role': 'user', 'content': "I'm in New York, NY."},
 <OpenAIObject at 0x7f7d00624bd0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"New York, NY\",\n  \"format\": \"celsius\"\n}"
   }
 }]

In [30]:
chat_response

<OpenAIObject chat.completion id=chatcmpl-8nDjCVnVfqjB37PXf107jCjmhsfTP at 0x7f7d0060f9c0> JSON: {
  "id": "chatcmpl-8nDjCVnVfqjB37PXf107jCjmhsfTP",
  "object": "chat.completion",
  "created": 1706741534,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New York, NY\",\n  \"format\": \"celsius\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 139,
    "completion_tokens": 27,
    "total_tokens": 166
  },
  "system_fingerprint": null
}

## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [31]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])

This sudden change in direction means we don't have enough time to complete the entire project for the client.


# Formatted output

In [32]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = openai.ChatCompletion.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 3.69 ms, sys: 1.72 ms, total: 5.42 ms
Wall time: 4.14 s


In [33]:
json_output = response["choices"][0]["message"]["content"]

In [34]:
output = json.loads(json_output)

In [35]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tablespoons granulated sugar',
 '1 1/2 teaspoons baking powder',
 '1/2 teaspoon salt',
 '3/4 cup milk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh blueberries',
 'Additional butter or oil for greasing the pan']

In [36]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, and salt.',
 'In a separate bowl, whisk together the milk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. Do not overmix; a few lumps are okay.',
 'Gently fold in the blueberries.',
 'Heat a non-stick griddle or skillet over medium heat. Grease with butter or oil.',
 'Pour 1/4 cup of batter onto the griddle for each pancake.',
 'Cook until bubbles form on the surface of the pancake and the edges start to look set, about 2-3 minutes.',
 'Flip the pancakes and cook for an additional 1-2 minutes, or until golden brown.',
 'Repeat with the remaining batter, adding more butter or oil to the pan as needed.',
 'Serve the blueberry pancakes warm with maple syrup and additional blueberries, if desired.']

# Translation

In [37]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [38]:
response["choices"][0]["message"]["content"]

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [39]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [40]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [41]:
print(response["choices"][0]["message"]["content"])

Fruit,Color,Taste
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour and bitter


In [42]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [43]:
print(response["choices"][0]["message"]["content"])

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>